In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load dataset
dataset = load_dataset("b-mc2/sql-create-context")

# Convert to Pandas DataFrame for splitting
df = pd.DataFrame(dataset["train"])

# Split into train and test (80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert back to Hugging Face dataset format
train_dataset = dataset["train"].select(train_df.index.tolist())
test_dataset = dataset["train"].select(test_df.index.tolist())

# Inspect dataset
print(f"Dataset example: {dataset['train'][0]}")

# Load tokenizer and model (T5 model for seq2seq tasks)
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)

# Data preprocessing function
def preprocess_function(examples):
    # Input: Natural language request + database schema
    inputs = [f"translate English to SQL: Schema: {context} Question: {question}" for context, question in zip(examples["context"], examples["question"])]

    # Output: SQL query
    targets = examples["answer"]

    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length", return_tensors="tf")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length", return_tensors="tf")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Preprocess dataset

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

# Create TensorFlow dataset
def to_tf_dataset(tokenized_dataset):
    return tf.data.Dataset.from_tensor_slices((
        {"input_ids": tokenized_dataset["input_ids"], "attention_mask": tokenized_dataset["attention_mask"]},
        tokenized_dataset["labels"]
    )).batch(4)

train_tf_dataset = to_tf_dataset(tokenized_train)
eval_tf_dataset = to_tf_dataset(tokenized_test)

# Compile model
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
model.compile(optimizer=optimizer)

# Train model
model.fit(train_tf_dataset, validation_data=eval_tf_dataset, epochs=3)

# Save model
model.save_pretrained("./sql-translator-model")
tokenizer.save_pretrained("./sql-translator-model")

# Testing function
def generate_sql(question, context):
    input_text = f"translate English to SQL: Schema: {context} Question: {question}"
    inputs = tokenizer(input_text, return_tensors="tf")

    outputs = model.generate(
        inputs["input_ids"],
        max_length=128,
        num_beams=4,
        early_stopping=True
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test example
test_question = "List all customer names and email addresses"
test_context = "CREATE TABLE customers (id INTEGER, name TEXT, surname TEXT, email TEXT, registration_date DATE);"

generated_sql = generate_sql(test_question, test_context)
print(f"Question: {test_question}")
print(f"Generated SQL: {generated_sql}")


Dataset example: {'answer': 'SELECT COUNT(*) FROM head WHERE age > 56', 'question': 'How many heads of the departments are older than 56 ?', 'context': 'CREATE TABLE head (age INTEGER)'}


All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Map:   0%|          | 0/62861 [00:00<?, ? examples/s]

Map:   0%|          | 0/15716 [00:00<?, ? examples/s]

Epoch 1/3
15716/15716 [==============================] - 3689s 232ms/step - loss: 0.1101 - val_loss: 0.0331
Epoch 2/3
15716/15716 [==============================] - 3653s 232ms/step - loss: 0.0407 - val_loss: 0.0261
Epoch 3/3
15716/15716 [==============================] - 3651s 232ms/step - loss: 0.0327 - val_loss: 0.0227
Question: List all customer names and email addresses
Generated SQL: SELECT SUM(surname), id FROM customers WHERE email TEXT = 'Registration_date'


In [ ]:
model.save_pretrained("./sql-translator-model")
tokenizer.save_pretrained("./sql-translator-model")

('./sql-translator-model/tokenizer_config.json',
 './sql-translator-model/special_tokens_map.json',
 './sql-translator-model/spiece.model',
 './sql-translator-model/added_tokens.json',
 './sql-translator-model/tokenizer.json')

In [ ]:
def generate_sql(question, context):
    input_text = f"translate English to SQL: Schema: {context} Question: {question}"
    inputs = tokenizer(input_text, return_tensors="tf")

    outputs = model.generate(
        inputs["input_ids"],
        max_length=128,
        num_beams=4,
        early_stopping=True
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test example
test_question = "What are the maximum and minimum budget of the departments?"
test_context = "CREATE TABLE department (budget_in_billions INTEGER)"

generated_sql = generate_sql(test_question, test_context)
print(f"Question: {test_question}")
print(f"Generated SQL: {generated_sql}")

Question: What are the maximum and minimum budget of the departments?
Generated SQL: SELECT MAX(budget_in_billions) FROM department


In [ ]:
pip install evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 12.8 MB/s eta 0:00:00


In [ ]:
import evaluate

# Load BLEU metric
bleu = evaluate.load("sacrebleu")

# Define evaluation function
def compute_bleu(model, tokenizer, dataset):
    references = []
    predictions = []

    for example in dataset:
        question = example["question"]
        context = example["context"]
        ground_truth = example["answer"]

        # Generate SQL query
        generated_sql = generate_sql(question, context)

        references.append([ground_truth])  # BLEU expects a list of references
        predictions.append(generated_sql)

    # Compute BLEU score
    bleu_score = bleu.compute(predictions=predictions, references=references)
    return bleu_score["score"]

# Compute and print BLEU score for train and validation datasets
train_bleu = compute_bleu(model, tokenizer, train_dataset)
test_bleu = compute_bleu(model, tokenizer, test_dataset)

print(f"Train BLEU Score: {train_bleu}")
print(f"Validation BLEU Score: {test_bleu}")

KeyboardInterrupt: 